# 机器学习
机器学习常用算法学习总结笔记

## 文本学习


### 基本问题
输入特征，若以单个单词作为输入，那么会因文本的长短不一，特征大小也不能统一

### 词袋模型  

- Bag-of-words  
>统计输入词出现的频率，构成向量。忽略一些stop word，比如：a, the, of。同时love、loves这种，单复数，统计主要词干，算一种。这种模型可以应用于其它文本

特点：  
1. 单词出现的顺序，对向量无影响  
2. 倾向于长文本，比如同一个文本一次处理，和复制同一个文本内容放入一个文本中，词频翻倍，对词向量有影响  
3. 不能处理复合词

- sklearn中的词袋模型  
>CountVectorizer

```python
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit([list ok ])
# 输出词频
print(vect.vocabulary_.get('word'))
```

- stop words
>low-information信息含量低的词语，比如：the，a，and，of  
higly-frequent word 出现次数异常的多，和内容没多大关系

    从中导入NTLK，使用stopword

```python
import nltk
nltk.download('stopwords')  # 下载词库
```

```python
from nltk.corpus import stopwords
sw = stopwords.words('english')
print(len(sw)) # 179
```

- stemmer
> 词干提取，合并近义词

```python
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer('english')
st.stem('word')
# 显示word对应词干
```
     

- TF-IDF
> TF，由词频确定权重；IDF，由词出现在所有文本中的频率逆向加权，即，出现频率少的权重反而高

## 贝叶斯
naive bayes

- GaussianNB高斯朴素贝叶斯
>$$P(A|B) = \frac{P(A,B)}{P(B)} = \frac{P(B|A)P(A)}{P(B)}= \frac{P(B|A)P(A)}{\sum _{i=1}^{n}P(B|A_i)}$$
P(A)为先验概率可根据大数定理，用样本频率来估计，  
P(A|B)为后验概率，  
P(B)用于归一化，与类标记无关,常数  
P(B|A)，条件概率，似然概率

```python
from sklearn.naive_bayes import GaussionNB
clt = GaussionNB()
```

### 原理探讨
> $$P(h|D) = \frac{P(D|h)P(h)}{P(D)}$$
h为假设，D为输入数据，如$(x_i,y_i)$：  
P(h|D),给的数据集D，正确标记的概率；P(D|h),给定假设规则，根据输入数据，求得到对应标签的概率  
P(h)：假设空间中h的先验概率；P(D)：数据集的先验概率

> 高斯分布，均值为$\mu$，方差为$\sigma^2$：$$f(x) = \frac{1}{\sqrt{2\pi \sigma^2}}\exp{\frac{-(x-\mu)^2}{2\sigma^2}}$$

> 贝叶斯决策论：基于概率、误判损失来选择最优标记

- 期望损失 条件风险  
$$R(c_i|x) = \sum _{j=1}^N \lambda_{ij}P(c_j|x)$$
对每个样本x最小化，总体风险也将最小化.在每个样本上选择使条件风险最小的样本标记
$$h(x) = argmin R(c|x)$$  
误判损失：$\lambda$ ,属于0/1损失函数，$$\lambda_{ij} = \begin{cases}0 &i = j\\1 & 其它\end{cases}$$
条件风险 $R(c|x) = 1-P(c|x)$

- 则贝叶斯最优分类器为$$h(x) = argmax P(c|x)$$

根据贝叶斯公式：
$$P(c|x) = \frac{P(x|c)P(c)}{P(x)}$$

难在P(x|c)，指已知假设标记c，在所有数据上标记正确的概率，难以从有限数据上得出，所以假设条件独立
$$P(x|c_k) = P(x1,x2,x3,x4...|c_k) = \prod_{i=1}^nP(x_i|c_k)$$,其中c_k可以看似c，就看作当分类为一种的时候好理解点

对于所有类别，P(x)相同，argmax P(c|x)，就等同于
$$argmaxP(c)\prod_{i=1}^nP(x_i|c_k)$$
P(x)相同的原因，全概率公式：
$$P(x) = \sum_i^kP(x|c_i)P(c_i)=\sum_i^kP(c_i)\prod_{j=1}^nP(x_j|c_i)$$

其中，先验概率P(c),充足的独立同分布样本，$P(c)=\frac{|D_c|}{D}$

离散属性：$P(x_i|c) = \frac{|D_{c,x_i}|}{|D|}$

连续属性，假定其属于高斯分布：$$P(x_i|c) = \frac{1}{\sqrt{2\pi}\sigma_{c,i}} \exp{\frac{-(x-\mu_{c,i})^2}{2\sigma_{c,i}^2}}$$

- 拉普拉斯平滑  
为避免属性之间有不同的组合，避免有的组合没有出现，但是算出的概率为0，被抹去，采用平滑处理。实际上假设属性、类别均分分布的  

N,代表对应标签的类别数：$$P(c)=\frac{|D_c|+1}{D+N}$$
N_i,代表属性i的可能的取值数：$$P(x_i|c) = \frac{|D_{c,x_i}|+1}{|D|+N_i}$$

### 应用

>优点：在数据较少的情况下仍然有效，可以处理多类别问题。
缺点：对于输入数据的准备方式较为敏感。
适用数据类型：标称型数据。

>朴素贝叶斯：假设特征同等重要，特征之间独立

## 决策树

### 最优特征选择

- 信息增益

信息熵entropy：度量样本集合D纯合度，值越小，纯度越高。
定义：$$Ent(D) = -\sum_{i=1}^N{P_i}{\log P_i}$$

信息增益information gain，集合D,属性a，a有V个取值，$D^v$表示第v个取值的样本数，由于样本数不同，增加权重$\frac{|D^v|}{|D|}$：
$$Gain(D,a) = Ent(D) - \sum_{v=1}^V\frac{|D^v|}{|D|}Ent(D^v)$$

信息增益越大，表示获得的提升越大，属性就选择使Gain(D,a)最大的值

缺点：对可取值数目多的属性有所偏好

- 信息增益比

信息增益率gain ratio：信息增益和关于属性a的熵之比
$$Gain_ratio(D,a) = \frac{Gain(D,a)}{IV(a)}$$

其中IV(a):$$IV(a) = - \sum_{v=1}^V\frac{|D^v|}{|D|}log \frac{|D^v|}{|D|}$$

样本数量越多，样本取值V越多，通常增益比降低，通常先计算增益，找出高于平均水平的，然后再从中选择增益比高的

- 基尼系数

基尼系数，gini index，从数据集D中随机抽取两个样本，类别标记不一致的概率，gini系数越小，纯度越高
$$Gini(D) = 1-\sum_{i=1}^{N}P^2_i$$

属性a的基尼系数为：$$Gini\_index = \sum_{i=1}^V\frac{|D^v|}{|D|}Gini(D^v)$$